**Assignment 3 - Facial Recognition Neural Network**


Data Preparation

In [2]:
import pandas as pd
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datapoints
from torchvision.transforms import ToTensor
from PIL import Image

from torchvision.transforms import RandomRotation, RandomHorizontalFlip, GaussianBlur

# Data augmentation class
data_augmentation = torch.nn.Sequential(
    # RandomRotation(degrees=15),
    RandomHorizontalFlip(p=0.3),
    # GaussianBlur(kernel_size=3)
)

train_target = pd.read_csv("/content/train_target.csv", header=None).to_numpy()


# Creating own testing data to prevent having to submit to kaggle for testing error
train_data = pd.read_csv("/content/train_data.csv", header=None).to_numpy()
train_data, test_dataA, train_target, test_targetA = train_test_split(
    train_data, train_target, test_size=0.2, random_state=42
)

# Converting training data/target to tensors and correct data-type
train_target = torch.from_numpy(train_target).to('cuda')
train_target = train_target.to(torch.int64)
train_data = torch.from_numpy(train_data).to('cuda')
train_data = train_data.to(torch.float32)


# Data Augmentation Attempted not used in final

# train_data = torch.nn.functional.normalize(train_data, dim=1)
# num_samples = train_data.size(0)
# train_data = train_data.view(num_samples, 1, 48, 48)
# train_data = torch.stack([data_augmentation(img) for img in train_data])
# train_data = train_data.view(num_samples, -1)


# Kaggle testing data provided 
test = pd.read_csv("/content/test_data.csv", header=None).to_numpy()
test_data = torch.from_numpy(test).to('cuda')
test_data = test_data.to(torch.float32)

# Data loader for training
dataloader = DataLoader(
    dataset=torch.utils.data.TensorDataset(train_data, train_target),
    batch_size=16,
    shuffle=True
)


# 1 x 48 x 48

# Image Classifier Model Class
class ImageClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        # convolutional layers
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(0.2),
        nn.MaxPool2d(kernel_size=2, stride=2),

        # fully connected layers
        nn.Flatten(),

        nn.Linear(64 * 6 * 6, 128),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.30),
        nn.BatchNorm1d(128),

        nn.Linear(128, 64),
        nn.LeakyReLU(0.2),
        nn.Dropout(0.30),
        nn.BatchNorm1d(64),

        nn.Linear(64, 3),  # output layer with 3 neurons for 3 classes
        nn.Softmax(dim=1)  # softmax to get the class probabilities
    )

  def forward(self, x):
    return self.model(x)

# Create instance, loss, and optimizer

lr = .001
epochs = 10

clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr)
loss_fn = nn.CrossEntropyLoss()

# Training loop

if __name__ == "__main__":
  for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        batch_size = X_batch.size(0)
        X_batch = X_batch.view(batch_size, 1, 48, 48)
        y_pred = clf(X_batch)
        y_batch = y_batch.squeeze()

        loss = loss_fn(y_pred, y_batch)

        opt.zero_grad()
        loss.backward()
        opt.step()

    print(f"Epoch {epoch} loss is {loss.item()}")



FileNotFoundError: [Errno 2] No such file or directory: '/content/train_target.csv'

In [197]:
test_dataA = torch.from_numpy(test_dataA).to('cuda')
test_dataA = test_dataA.to(torch.float32)
batch_size = test_dataA.size(0)
test_dataA = test_dataA.view(batch_size, 1, 48, 48)
test_dataA = test_dataA.to(torch.float32)
test_targetA = torch.from_numpy(test_targetA).to('cuda')

In [202]:

test_predA = clf(test_dataA)
test_targetA = test_targetA.squeeze()
loss = loss_fn(test_predA, test_targetA)
print(loss)


tensor(0.8083, device='cuda:0', grad_fn=<NllLossBackward0>)


In [204]:
# Processing Kaggle test data
batch_size = test_data.size(0)
test_data = test_data.view(batch_size, 1, 48, 48)
test_data = test_data.to(torch.float32)

# Formatting output to Kaggle format Id:Category
test_pred = clf(test_data)
predicted_classes = torch.argmax(test_pred, dim=1).cpu().numpy()
df = pd.DataFrame({'Id': range(len(predicted_classes)), 'Category': predicted_classes})
print(df)

# Output save as file
filepath = "/content/predictions.csv"
df.to_csv(filepath, index=False)

        Id  Category
0        0         0
1        1         0
2        2         1
3        3         1
4        4         1
...    ...       ...
3960  3960         0
3961  3961         1
3962  3962         1
3963  3963         2
3964  3964         2

[3965 rows x 2 columns]


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0